# The Diet Problem

The goal of the diet problem is to find the cheapest combination of foods that will satisfy all the daily nutritional requirements of a person.
The diet problem is one of the first optimization problems to be studied back in the 1930's and 40's. It was first motivated by the Army desire to meet the nutritional requirements of the soldier while minimizing the cost. 


# Mathematical Model


The problem is formulated as a linear program where the objective is to minimize cost and meet constraints which require that nutritional needs be satisfied. They include constraints that regulate the number of calories and amounts of vitamins, minerals, fats, sodium and cholesterol in the diet.  

Let us introduce the following notations:

m-number of nutrients, 
n-number of foods
C - vector of food unit cost
$$c=(c_1,..c_n)$$
matrix A - amount of nutrient j in food i
$$A=(a_{ij})$$ 
b-vector of daily nutritional requirements
$$b=(b_1,..b_m)$$
X- decision vector ( numbers of servings of each food in daily diet)
$$X=(x_1,..x_n)$$ 

Then $\sum_{i=1}^n{c_i x_i}$ is a cost of daily diet

$\sum_{i=1}^n{a_{ij} x_i}$ is the amount of nutrient j in daily diet

the optimization problem is formulated as follows:

objective - minimize cost of daily diet:
$$\min_x(\sum_{i=1}^n{c_i x_i})$$

constraints: amounts of nutrients in daily diet should satisfy daily nutritional requirements:
$$\sum_{i=1}^n{a_{ij} x_i}>=b_j, j=1,m$$
bounds on decision variables: number of servinds of each food should not be negative
$$x_i>=0, i=1,n$$





# Python solver

The above optimization problem ( linear programming problem) can be solved with Python linprog solver from Scipy module (scipy.optimize.linprog). This is a description of linprog functionality:

Linear programming: minimize a linear objective function subject to linear equality and inequality constraints.

scipy.optimize.linprog(c, A_ub=None, b_ub=None, A_eq=None, b_eq=None, bounds=None, method='highs', callback=None, options=None, x0=None, integrality=None

this is a link to full description in scipy documentatation:
https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.linprog.html

We will create a python script that loads input data from Excel spreadsheet, transforms it to a form suitable for scipy.optimize.linprog solver, runs scipy.optimize.linprog solver and prints solver output



We use pandas read_excel function to read input data

In [2]:
import pandas as pd

we use numpy to perform vector and matrix operations 

In [3]:
import numpy as np

we use scipy.optimize.linpprog solver to solve linear programming problem

In [4]:
import scipy.optimize as spo

this is a spreadsheet with input data

In [5]:
data_file = 'DietDataFormatted.xlsx'

From this input data we need to prepare matrices and vectors (A,b,c) for linprog input.

First, read nutritional info for each food that we want to be in our daily diet.

In [10]:
df_A = pd.read_excel(data_file,sheet_name='NutritionalInfo',header = 0,index_col = 0)

In [11]:
df_A

,fat,vitA,vitC,protein,calories
broccoli,0.8,70.0,160.2,8.0,74.0
apple,0.5,73.1,7.9,0.3,81.4
oatm.cookie,3.3,2.9,0.1,1.1,81.0
milk,4.7,100.0,2.3,8.1,121.2
chcken,10.8,77.4,2.0,42.2,227.2
omelette,7.3,409.2,0.1,6.7,99.6


convert this DataFrame into numpy array A 

In [12]:
A = np.transpose(df_A.to_numpy())

In [13]:
A

array([[8.000e-01, 5.000e-01, 3.300e+00, 4.700e+00, 1.080e+01, 7.300e+00],
       [7.000e+01, 7.310e+01, 2.900e+00, 1.000e+02, 7.740e+01, 4.092e+02],
       [1.602e+02, 7.900e+00, 1.000e-01, 2.300e+00, 2.000e+00, 1.000e-01],
       [8.000e+00, 3.000e-01, 1.100e+00, 8.100e+00, 4.220e+01, 6.700e+00],
       [7.400e+01, 8.140e+01, 8.100e+01, 1.212e+02, 2.272e+02, 9.960e+01]])

read daily requirements info for each nutrient

In [14]:
df_b = pd.read_excel(data_file,sheet_name='Requirements',header = 0,index_col = 0)

In [15]:
df_b

,fat,vitA,vitC,protein,calories
lower bounds,20,50,50,50,1000
upper bounds,65,600,300,110,2000


from this data create two numy array, b and b_lower

In [16]:
b = df_b.to_numpy()[1]
b_lower = -df_b.to_numpy()[0]

In [17]:
b

array([  65,  600,  300,  110, 2000], dtype=int64)

In [18]:
b_lower

array([  -20,   -50,   -50,   -50, -1000], dtype=int64)

read food unit costs

In [19]:
df_c = pd.read_excel(data_file,sheet_name='UnitCosts',header = None,index_col = 0)

In [20]:
df_c

,1
0,
broccoli,0.16
apple,0.24
oatm.cookie,0.09
milk,0.23
chcken,0.84
omelette,0.11


create numpy array from unit costs data

In [21]:
c = df_c.to_numpy().flatten()

In [22]:
c

array([0.16, 0.24, 0.09, 0.23, 0.84, 0.11])

In [24]:
A_lower = -A
A_ub = np.vstack([A,A_lower])
b_ub = np.vstack([b,b_lower])

create bounds for desition variables

In [25]:
bounds = (0, 4)
bounds1 = (0, 3)
bounds2 = (0, 7)
bounds = (bounds,bounds,bounds,bounds1,bounds,bounds2)

In [26]:
bounds

((0, 4), (0, 4), (0, 4), (0, 3), (0, 4), (0, 7))

nput matrices and vectors A,b,c are ready, we can run linprog solver

In [28]:
res=spo.linprog(c,A_ub=A_ub,b_ub=b_ub,bounds=bounds)

the result of optimization is a complex object that contains information about optimal vaue of objective function (daily diet cost), values of decision variables (optimal number of portions for each food), and the optimzation process info

In [29]:
res

     fun: 1.8837741420058
 message: 'Optimization terminated successfully.'
     nit: 11
   slack: array([  33.30997776,    0.        ,    0.        ,   60.        ,
       1000.        ,    2.26124526,  550.        ,  250.        ,
          0.        ,    3.84774898,    6.93442043,    2.24804677,
          0.        ,    0.        ,    0.        ,    0.        ])
  status: 0
 success: True
       x: array([1.73875474, 1.75195323, 4.        , 3.        , 0.15225102,
       0.06557957])

from the result object we print daily meal overview as follows

In [30]:
print ('**********************************************************')   
print ('cost of daily meal, optimized:', res.fun)
print ('**********************************************************')   
print ('here is the meal plan ( number of servings of each food):')
for i in range(len(df_A.index)):
    print (df_A.index[i], ':', res.x[i])

print ('**********************************************************')    
print ('you are getting these nutrients:')    
for j in range(len(df_A.columns)):
    nutrient_j = 0.0
    for i in range(len(df_A.index)):
        nutrient_j += A[j,i]*res.x[i]
    print (df_A.columns[j] , ':', nutrient_j)    

**********************************************************
cost of daily meal, optimized: 1.8837741420058
**********************************************************
here is the meal plan ( number of servings of each food):
broccoli : 1.7387547409077408
apple : 1.7519532292895434
oatm.cookie : 4.0
milk : 3.0
chcken : 0.1522510193240766
omelette : 0.06557956544405913
**********************************************************
you are getting these nutrients:
fat : 31.69002224381262
vitA : 600.0
vitC : 299.99999999999994
protein : 50.000000000000014
calories : 1000.0000000000001
